In [1]:
import numpy as np
import cirq 

In [29]:
def make_bb84_circ(num_qubits, alice_basis, bob_basis, alice_state):
    
    qubits = [ cirq.LineQubit(i) for i in range(num_qubits) ]
    circuit = cirq.Circuit()
    
    #Alice prepares qubits
    alice_enc = []
    for index, i in enumerate(alice_basis):
        if alice_state[index] == 1:
            alice_enc.append(cirq.X(qubits[index]))
        if alice_basis[index] == 1:
            alice_enc.append(cirq.H(qubits[index]))
    
    circuit.append(alice_enc)
    
    #Bob measures received qubits
    bob_basis_choice = []
    for index, i in enumerate(bob_basis):
        if bob_basis == 1:
            bob_basis_choice.append(cirq.H(qubits[index]))
    
    circuit.append(bob_basis_choice)
    circuit.append(cirq.measure_each(*qubits))
    
    return circuit

In [30]:
def bitstrings(bits):
    return ''.join(str(int(b)) for b in bits)

In [67]:
def print_results(alice_basis, bob_basis, alice_state, expected_key, obtained_key):
    num_qubits = len(alice_basis)
    basis_match = ''.join(
            ['X' if alice_basis[i] == bob_basis[i] else '_' for i in range(num_qubits)])
    
    alice_basis_str = ''.join(['C' if alice_basis[i] == 0 else 'H' for i in range(num_qubits)])
    bob_basis_str = ''.join(['C' if bob_basis[i] == 0 else 'H' for i in range(num_qubits)])
    
    print(f"Alice basis: {alice_basis_str}")
    print(f"Bob basis: {bob_basis_str}")
    print(f"Alice bits: {bitstrings(alice_state)}")
    print(f"Bases match: {basis_match}")
    print(f"Expected key: {expected_key}")
    print(f"Actual key: {obtained_key}")

In [109]:
def main(num_qubits = 8):
    #set up non-eavesdropped protocol
    print("Simulating non-eavesdropped protocol")
    alice_basis = [np.random.randint(0,2) for i in range(num_qubits)]
    alice_state = [np.random.randint(0,2) for i in range(num_qubits)]
    bob_basis = [np.random.randint(0,2) for i in range(num_qubits)]
    
    expected_key = bitstrings(
            [alice_state[i] for i in range(num_qubits) if alice_basis[i] == bob_basis[i]])
    
    circuit = make_bb84_circ(num_qubits, alice_basis, bob_basis, alice_state)
    
    #run simulations
    repetitions = 1
    result = cirq.Simulator().run(program = circuit, repetitions=repetitions)
    result_bitstring = bitstrings([int(result.measurements[str(i)]) for i in range(num_qubits)])
    
    #take only qubits where bases match
    obtained_key = ''.join(
            [result_bitstring[i] for i in range(num_qubits) if alice_basis[i] == bob_basis[i]])
    
    assert expected_key == obtained_key, "Keys don't match"
    print(circuit)
    print_results(alice_basis, bob_basis, alice_state, expected_key, obtained_key)
    
    #set up eavesdropped protocol
    print("Simulating eavesdropped protocol")
    alice_basis = [np.random.randint(0,2) for i in range(num_qubits)]
    alice_state = [np.random.randint(0,2) for i in range(num_qubits)]
    bob_basis = [np.random.randint(0,2) for i in range(num_qubits)]
    eve_basis = [np.random.randint(0,2) for i in range(num_qubits)]
    
    expected_key = bitstrings(
            [alice_state[i] for i in range(num_qubits) if alice_basis[i] == bob_basis[i]])
    
    #Eve intercepts qubits
    alice_eve_circuit = make_bb84_circ(num_qubits, alice_basis, eve_basis, alice_state)
    
    #run simulations
    repetitions = 1
    result = cirq.Simulator().run(program=alice_eve_circuit, repetitions = repetitions)
    eve_state = [int(result.measurements[str(i)]) for i in range(num_qubits)]
    
    eve_bob_circuit = make_bb84_circ(num_qubits, eve_basis, bob_basis, eve_state)
    
    #run simulations
    repetitions = 1
    result = cirq.Simulator().run(program=eve_bob_circuit, repetitions=repetitions)
    result_bitstring = bitstrings([int(result.measurements[str(i)]) for i in range(num_qubits)])
    
    #take only qubits where bases match
    obtained_key = ''.join(
            [result_bitstring[i] for i in range(num_qubits) if alice_basis[i] == bob_basis[i]])
    
    assert expected_key != obtained_key, "Keys shouldn't match"
    
    circuit = alice_eve_circuit + eve_bob_circuit
    print(circuit)
    print_results(alice_basis, bob_basis, alice_state, expected_key, obtained_key)

In [110]:
main()

Simulating non-eavesdropped protocol
0: ───X───H───M───

1: ───M───────────

2: ───M───────────

3: ───X───M───────

4: ───M───────────

5: ───M───────────

6: ───H───M───────

7: ───X───M───────
Alice basis: HCCCCCHC
Bob basis: HHHHCHCC
Alice bits: 10010001
Bases match: X___X__X
Expected key: 101
Actual key: 101
Simulating eavesdropped protocol
0: ───X───H───M───X───H───M───

1: ───X───H───M───X───H───M───

2: ───X───M───────X───M───────

3: ───H───M───────H───M───────

4: ───X───M───────X───H───M───

5: ───X───H───M───M───────────

6: ───H───M───────X───M───────

7: ───M───────────M───────────
Alice basis: HHCHCHHC
Bob basis: HHCCCHCC
Alice bits: 11101100
Bases match: XXX_XX_X
Expected key: 111110
Actual key: 111100


In [68]:
num_qubits = 8

In [69]:
#set up non-eavesdropped protocol
alice_basis = [np.random.randint(0,2) for i in range(num_qubits)]
alice_state = [np.random.randint(0,2) for i in range(num_qubits)]
bob_basis = [np.random.randint(0,2) for i in range(num_qubits)]

expected_key = bitstrings(
    [ alice_state[i] for i in range(num_qubits) if alice_basis[i] == bob_basis[i] ])

expected_key

'11'

In [70]:
circuit = make_bb84_circ(num_qubits, alice_basis, bob_basis, alice_state)
circuit

0: ───X───M───────

1: ───M───────────

2: ───X───M───────

3: ───X───M───────

4: ───X───H───M───

5: ───X───M───────

6: ───H───M───────

7: ───X───H───M───

In [71]:
repetitions = 1

result = cirq.Simulator().run(program=circuit, repetitions = repetitions)
result_bitstring = bitstrings([int(result.measurements[str(i)]) for i in range(num_qubits)])

result_bitstring

'10110101'

In [72]:
obtained_key = ''.join(
    result_bitstring[i] for i in range(num_qubits) if alice_basis[i] == bob_basis[i])

obtained_key

'01'

In [73]:
assert expected_key == obtained_key, "Keys don't match"

AssertionError: Keys don't match

In [74]:
print_results(alice_basis, bob_basis, alice_state, expected_key, obtained_key)

Alice basis: CCCCHCHH
Bob basis: HHHHHHCH
Alice bits: 10111101
Bases match: ____X__X
Expected key: 11
Actual key: 01


In [82]:
#set up eavesdropped protocol
#np.random.seed(200)

alice_basis = [np.random.randint(0,2) for i in range(num_qubits)]
alice_state = [np.random.randint(0,2) for i in range(num_qubits)]
bob_basis = [np.random.randint(0,2) for i in range(num_qubits)]
eve_basis = [np.random.randint(0,2) for i in range(num_qubits)]

expected_key = bitstrings(
                [alice_state[i] for i in range(num_qubits) if alice_basis[i] == bob_basis[i]])

expected_key

'0100'

In [83]:
alice_eve_circuit = make_bb84_circ(num_qubits, alice_basis, eve_basis, alice_state)

alice_eve_circuit

0: ───M───────

1: ───H───M───

2: ───M───────

3: ───X───M───

4: ───M───────

5: ───X───M───

6: ───H───M───

7: ───M───────

In [93]:
repetitions = 1
result = cirq.Simulator().run(program=alice_eve_circuit, repetitions = repetitions)
eve_state = [int(result.measurements[str(i)]) for i in range(num_qubits)]

eve_state

[0, 0, 0, 1, 0, 1, 1, 0]

In [94]:
eve_bob_circuit = make_bb84_circ(num_qubits, eve_basis, bob_basis, eve_state)

eve_bob_circuit

0: ───M───────────

1: ───H───M───────

2: ───H───M───────

3: ───X───M───────

4: ───H───M───────

5: ───X───H───M───

6: ───X───H───M───

7: ───H───M───────

In [99]:
repetitions = 1
result = cirq.Simulator().run(program=eve_bob_circuit, repetitions=repetitions)
result_bitstring = bitstrings([int(result.measurements[str(i)]) for i in range(num_qubits)])

result_bitstring

'01010111'

In [100]:
assert expected_key != obtained_key, "Keys shouldn't match"

In [102]:
circuit = alice_eve_circuit + eve_bob_circuit
circuit

0: ───M───────M───────────

1: ───H───M───H───M───────

2: ───M───────H───M───────

3: ───X───M───X───M───────

4: ───M───────H───M───────

5: ───X───M───X───H───M───

6: ───H───M───X───H───M───

7: ───M───────H───M───────